## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Atuona,-9.80,-139.03,78.19,77,14,14.56,light rain
1,1,Khatanga,71.97,102.50,1.63,92,100,4.88,overcast clouds
2,2,Rock Springs,41.59,-109.20,55.40,22,1,12.75,clear sky
3,3,Baykit,61.67,96.37,18.48,94,100,5.28,light snow
4,4,Tuktoyaktuk,69.45,-133.04,6.80,92,75,10.29,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Atuona,-9.80,-139.03,78.19,77,14,14.56,light rain
8,8,Georgetown,5.41,100.34,80.60,83,20,3.36,few clouds
29,29,Hilo,19.73,-155.09,77.00,69,90,4.70,overcast clouds
32,32,Wattegama,6.80,81.48,80.60,94,90,3.36,overcast clouds
33,33,Hithadhoo,-0.60,73.08,83.39,73,100,11.21,light rain
36,36,Bathsheba,13.22,-59.52,86.00,74,40,21.92,shower rain
38,38,Buala,-8.14,159.59,80.74,78,100,4.00,light rain
44,44,Vallenar,-28.57,-70.76,82.90,15,0,10.45,clear sky
46,46,Butaritari,3.07,172.79,81.68,78,1,10.40,clear sky
49,49,Praia,14.92,-23.51,84.20,48,40,11.41,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                193
City                   193
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Weather Description    193
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df[["City", "Max Temp", "Weather Description","Lat", "Lng"]].copy()
clean_df.dropna()
clean_df.head(10)

,City,Max Temp,Weather Description,Lat,Lng
0,Atuona,78.19,light rain,-9.80,-139.03
8,Georgetown,80.60,few clouds,5.41,100.34
29,Hilo,77.00,overcast clouds,19.73,-155.09
32,Wattegama,80.60,overcast clouds,6.80,81.48
33,Hithadhoo,83.39,light rain,-0.60,73.08
36,Bathsheba,86.00,shower rain,13.22,-59.52
38,Buala,80.74,light rain,-8.14,159.59
44,Vallenar,82.90,clear sky,-28.57,-70.76
46,Butaritari,81.68,clear sky,3.07,172.79
49,Praia,84.20,scattered clouds,14.92,-23.51


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Max Temp", "Weather Description","Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Atuona,78.19,light rain,-9.80,-139.03,
8,Georgetown,80.60,few clouds,5.41,100.34,
29,Hilo,77.00,overcast clouds,19.73,-155.09,
32,Wattegama,80.60,overcast clouds,6.80,81.48,
33,Hithadhoo,83.39,light rain,-0.60,73.08,
36,Bathsheba,86.00,shower rain,13.22,-59.52,
38,Buala,80.74,light rain,-8.14,159.59,
44,Vallenar,82.90,clear sky,-28.57,-70.76,
46,Butaritari,81.68,clear sky,3.07,172.79,
49,Praia,84.20,scattered clouds,14.92,-23.51,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df

,City,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Atuona,78.19,light rain,-9.80,-139.03,Villa Enata
8,Georgetown,80.60,few clouds,5.41,100.34,Cititel Penang
29,Hilo,77.00,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
32,Wattegama,80.60,overcast clouds,6.80,81.48,
33,Hithadhoo,83.39,light rain,-0.60,73.08,Scoop Guest House
...,...,...,...,...,...,...
687,Narasannapeta,76.86,broken clouds,18.42,84.05,G.lingaraju home
689,Kandi,81.09,clear sky,11.13,2.94,Maison des producteurs kandi
691,Mandera,82.20,moderate rain,3.94,41.87,Kornesh Hotel and Lodging
692,Assomada,84.20,scattered clouds,15.10,-23.68,Residencial Cosmos


In [28]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isna()

,City,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,False,False,False,False,False,False
8,False,False,False,False,False,False
29,False,False,False,False,False,False
32,False,False,False,False,False,False
33,False,False,False,False,False,False
...,...,...,...,...,...,...
687,False,False,False,False,False,False
689,False,False,False,False,False,False
691,False,False,False,False,False,False
692,False,False,False,False,False,False


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))